In [ ]:
# test portfolio : {ticker : ratio(graham_number/price)
test_dictionary = {'MSFT': 0.13963799348298295,
                   'PEP': 0.12020944922261735,
                   'AAPL': 0.03260789823106461}


In [20]:
import pandas as pd
import zipfile


In [21]:
# get data

ticker = 'MSFT'

def get_key_metrics(zip_file_path: str, ticker: str):
    """
    :param zip_file_path: file path of the zip folder the key_metrics.json file is stored in
    :param ticker: ticker symbol as string
    :return: pandas dataframe containing key metrics for the given ticker symbol
    """
    file_name = f'{ticker}_key-metrics.json'
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        with zip_ref.open(file_name) as file:
            content = pd.read_json(file)
            return content

key_metrics = get_key_metrics('key-metrics_3-Aktien.zip', ticker)
print(key_metrics)
print(key_metrics.columns)

    symbol       date  calendarYear period  revenuePerShare  \
0     MSFT 2023-09-30          2024     Q1         7.580899   
1     MSFT 2023-06-30          2023     Q4         7.526738   
2     MSFT 2023-03-31          2023     Q3         7.076870   
3     MSFT 2022-12-31          2023     Q2         7.065047   
4     MSFT 2022-09-30          2023     Q1         6.709199   
..     ...        ...           ...    ...              ...   
148   MSFT 1986-09-30          1987     Q1         0.009098   
149   MSFT 1986-06-30          1986     Q4         0.008705   
150   MSFT 1986-03-31          1986     Q3         0.007102   
151   MSFT 1985-12-31          1986     Q2         0.005985   
152   MSFT 1985-09-30          1986     Q1         0.005985   

     netIncomePerShare  operatingCashFlowPerShare  freeCashFlowPerShare  \
0             2.990000                   4.102246              2.772031   
1             2.689929                   3.853855              2.655905   
2             2.45

In [26]:

# print(key_metrics.columns)
for index, row in key_metrics.iterrows():
    if row['calendarYear'] == 2024:
        print(row)

symbol                                MSFT
date                   2023-09-30 00:00:00
calendarYear                          2024
period                                  Q1
revenuePerShare                   7.580899
                              ...         
receivablesTurnover               1.529429
payablesTurnover                  0.844357
inventoryTurnover                    5.434
roe                               0.100995
capexPerShare                    -1.330215
Name: 0, Length: 61, dtype: object


In [22]:
from datetime import datetime
from credentials import api_key
# get closing price

def get_nearest_date(items, pivot):
    # Convert the date strings to datetime objects
    items = [datetime.strptime(date, "%Y-%m-%d") for date in items]
    pivot = datetime.strptime(pivot, "%Y-%m-%d")

    nearest = min(items, key=lambda x: abs(x - pivot))
    timedelta = abs(nearest - pivot)
    return nearest, timedelta

def get_closing_price(ticker: str, target_date: str):
    closing_price = None
    alpha_vantage_url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={ticker}&outputsize=full&apikey={api_key}&datatype=csv'
    alpha_vantage_csv = pd.read_csv(alpha_vantage_url)
    alpha_vantage_df = pd.DataFrame(alpha_vantage_csv)

    # Try fetching from Alpha Vantage
    for index, row in alpha_vantage_df.iterrows():
        if row['timestamp'] == target_date:
            closing_price = row['adjusted_close']
            print(f"Found exact match in Alpha Vantage: {closing_price}")
            return closing_price

    if closing_price is None:
        # If both fail, find the nearest closing price
        dates = alpha_vantage_df['timestamp']
        nearest_date, timedelta = get_nearest_date(dates, target_date)

        # Get closing price of nearest date
        nearest_date_str = nearest_date.strftime("%Y-%m-%d")
        for index, row in alpha_vantage_df.iterrows():
            if row['timestamp'] == nearest_date_str:
                nearest_closing_price = row['adjusted_close']
                print(f"Found nearest closing price: {nearest_closing_price} on {nearest_date_str}")
                return nearest_closing_price
    else:
        return closing_price

In [23]:
start_evaluation_date = '2019-09-30'
print(start_evaluation_date)
price_data_start = get_closing_price('MSFT', start_evaluation_date)
print(price_data_start)


end_evaluation_date = '2022-09-30'
price_data_end = get_closing_price('MSFT', end_evaluation_date)
print(end_evaluation_date)
print(price_data_end)

2019-09-30
Found exact match in Alpha Vantage: 133.503975950862
133.503975950862
Found exact match in Alpha Vantage: 230.204543223028
2022-09-30
230.204543223028


In [19]:
# shares
from credentials import api_key
import pandas as pd
import requests

def get_number_of_shares_outstanding(ticker, date):
    url = f'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol={ticker}&outputsize=full&apikey={api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'quarterlyReports':
            df = pd.DataFrame(value)

    for index, row in df.iterrows():
        if row['fiscalDateEnding'] == date:
            shares_outstanding = row['commonStockSharesOutstanding']
            return shares_outstanding


ticker = 'MSFT'
start_date = '2019-09-30'
shares_start = get_number_of_shares_outstanding(ticker, start_date)
print(shares_start)

end_date = '2022-09-30'
shares_end = get_number_of_shares_outstanding(ticker, end_date)
print(shares_end)

7634000000
7457000000


In [26]:
# sharesOutstandingStart * x = sharesOustandingEnd
# x = sharesOutstandingEnd / sharesOustandingStart

factor = int(shares_end) / int(shares_start)
print(factor)

0.9768142520303904


In [27]:
# now the new price that has to be considered for the evaluation is...

new_price = price_data_start / factor
print(new_price)

136.67283792528906


In [31]:
# thoughts on the calculation

# periodenrendite (siehe lüscher-marty BAND 2 S.22)

# periodenrendite = (Endkapital / Anfangskapital) -1
# = periodenrendite = Kursveränderung / Anfangskapital

# periodenrendite = (endwert - startwert) / startwert
# dividend per Share (over the period) = (end_price - start_price) / start_price

# startwert = new_price

dividend = (price_data_end - new_price) / new_price
print(dividend)
dividend_percent = dividend * 100
print(f'investing on the {start_date} for a price of {new_price} one would make an earning of {dividend_percent} % per share till {end_date}')


0.6843474293616935
investing on the 2019-09-30 for a price of 136.67283792528906 one would make an earning of 68.43474293616934 % per share till 2022-09-30


### make this functional

In [42]:
# imports
import zipfile
from datetime import datetime
from credentials import api_key
import requests

def get_closing_price(ticker: str, target_date: str):
    alpha_vantage_url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={ticker}&outputsize=full&apikey={api_key}&datatype=csv'
    alpha_vantage_csv = pd.read_csv(alpha_vantage_url)
    alpha_vantage_df = pd.DataFrame(alpha_vantage_csv)

    # Try fetching from Alpha Vantage
    for index, row in alpha_vantage_df.iterrows():
        if row['timestamp'] == target_date:
            closing_price = row['adjusted_close']
            print(f"Found exact match in Alpha Vantage: {closing_price}")
            return closing_price

# shares
def get_number_of_shares_outstanding(ticker, date):
    url = f'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol={ticker}&outputsize=full&apikey={api_key}'
    r = requests.get(url)
    data = r.json()
    for key, value in data.items():
        if key == 'quarterlyReports':
            df = pd.DataFrame(value)
            for index, row in df.iterrows():
                if row['fiscalDateEnding'] == date:
                    shares_outstanding = row['commonStockSharesOutstanding']
                    return shares_outstanding

def calculate_dividend(ticker, start, end):
    # get the number of shares outstanding
    shares_start = get_number_of_shares_outstanding(ticker, start)
    print(f'Shares Outstanding on start date: {shares_start}')
    shares_end = get_number_of_shares_outstanding(ticker, end)
    print(f'Shares Outstanding on end date: {shares_end}')


    # get the closing prices
    price_data_start = get_closing_price(ticker, start)
    print(f'price on start date: {price_data_start}')

    price_data_end = get_closing_price(ticker, end)
    print(f'price on end date: {price_data_end}')

    # adjust closing price
    factor = int(shares_end) / int(shares_start)
    adjusted_price = price_data_start / factor
    print(f'adjusted price on start date: {adjusted_price}')

    # calculate dividend
    dividend = (price_data_end - adjusted_price) / adjusted_price

    return dividend

In [43]:
# testing ...
# test portfolio : {ticker : ratio(graham_number/price)
test_dictionary = {'MSFT': 0.13963799348298295,
                   #'PEP': 0.12020944922261735,
                   'AAPL': 0.03260789823106461}

ticker_list = list(test_dictionary.keys())

start_date = '2019-09-30'
end_date = '2022-09-30'

for ticker in ticker_list:
    print(ticker)
    print(calculate_dividend(ticker, start_date, end_date))

MSFT
Shares Outstanding on start date: 7634000000
Shares Outstanding on end date: 7457000000
Found exact match in Alpha Vantage: 133.503975950862
price on start date: 133.503975950862
Found exact match in Alpha Vantage: 230.204543223028
price on end date: 230.204543223028
adjusted price on start date: 130.4085864115245
0.7652560276712292
AAPL
Shares Outstanding on start date: 4443236000
Shares Outstanding on end date: 15943425000
Found exact match in Alpha Vantage: 54.389066336395
price on start date: 54.389066336395
Found exact match in Alpha Vantage: 137.207418672583
price on end date: 137.207418672583
adjusted price on start date: 195.1613643646969
-0.29695398923229344


### Probleme:

- die werte können eigentlich nicht stimmen (v.a. Apple 800 % Dividende ?!)
- nicht möglich ein datum festzulegen muss irgendwie wieder über das quartal und jahr festgelegt werden
- und dann ggf. das nächstegelegene datum gesucht werden...
--> target date von graham number berechnung (das wird als startwert genommen und dann einfach drei jahre später das nächste target date definieren...)